### Contents
    01. Importing Libraries and Data 
    02. Data checks and preparation
    03. PII issues
    04. Create flags for regions
    05. Spending and ordering habits across regions¶
    06. Creating dataframe to show aggregate values for regions
    07. Excluding low frequency customers
    08. Export

# Import Libraries 

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# Import Data 

In [2]:
# set path
path=r'C:\Users\niels\Documents\Master Folder - Instacart Basket Analysis 09.10'

In [3]:
# Import latest version of merged data 
df = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'full_instacart_data.pkl'))

# Data Checks 

In [4]:
# Check df 
df.head()

,Order_id,Customer_id,Order_number,Orders_day_of_week,Order_time_of_day,Days_since_prior_order,Product_id,Add_to_cart_order,Reordered,Product_name,...,First_name,Surname,Gender,State,Age,Date_joined,#_of_dependants,Marital_status,Income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [5]:
df.shape

(32404859, 33)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 33 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   Order_id                int32   
 1   Customer_id             object  
 2   Order_number            int8    
 3   Orders_day_of_week      int8    
 4   Order_time_of_day       int8    
 5   Days_since_prior_order  float16 
 6   Product_id              int32   
 7   Add_to_cart_order       int32   
 8   Reordered               int8    
 9   Product_name            object  
 10  aisle_id                int64   
 11  department_id           int64   
 12  prices                  float64 
 13  _Merge                  category
 14  Busiest_day             object  
 15  Busiest_2_days          object  
 16  Busiest_period_of_day   object  
 17  Max_order               int8    
 18  Loyalty_flag            object  
 19  Mean_order_price        float64 
 20  Spending_flag           object  
 21  Order_

# Consider Security Implications 

For security purposes, there is no need to have customer names. Customer_id should be sufficient. 

In [7]:
# Drop the first and last name columns. The columns _merge and aisle_id can also be dropped while at it. 
df2 = df.drop(columns=['First_name', 'Surname', '_Merge', 'aisle_id','_merge']) 

In [8]:
# Check to makes sure successfully dropped 
df2.head()

,Order_id,Customer_id,Order_number,Orders_day_of_week,Order_time_of_day,Days_since_prior_order,Product_id,Add_to_cart_order,Reordered,Product_name,...,Spending_flag,Order_frequency,Frequency_flag,Gender,State,Age,Date_joined,#_of_dependants,Marital_status,Income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


# Create flags for regions

In [9]:
# create series for each region with all appropriate states
Northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 
             'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio',
          'North Dakota', 'South Dakota', 'Nebraska', 'Kansas',
          'Minnesota', 'Iowa', 'Missouri']
South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia',
         'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee',
         'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona',
        'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [10]:
# create a new region column identifying 
df2.loc[df2['State'].isin(Northeast), 'Region'] = 'Northeast'

In [11]:
df2.loc[df2['State'].isin(Midwest), 'Region'] = 'Midwest'

In [12]:
df2.loc[df2['State'].isin(South), 'Region'] = 'South'

In [13]:
df2.loc[df2['State'].isin(West), 'Region'] = 'West'

In [14]:
# make sure all records were allocated to a region
df2['Region'].value_counts(dropna=False)

South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: Region, dtype: int64

In [15]:
10791885+8292913+7597325+5722736

32404859

All records were successfully allocated to a region. 

# Spending and ordering habits across regions

In [16]:
# Group by region and aggregate by customer id in order to see which region has more customers
df2[['Region', 'Customer_id']].groupby('Region').agg({'Customer_id': pd.Series.nunique,}).sort_values(by='Customer_id',
                                                                                              ascending=False)

,Customer_id
Region,
South,68737
West,52565
Midwest,48519
Northeast,36388


The South has the most customers and the Northeast has the least.

In [17]:
# Important to note that the South also has the most states alloted to it's region 
df2[['Region', 'State']].groupby('Region').agg({'State': pd.Series.nunique,}).sort_values(by='State',
                                                                                          ascending=False)

,State
Region,
South,17
West,13
Midwest,12
Northeast,9


The South also has the most states and the Northeast has the least.

In [18]:
# Check to see which region has sold the most products 
df2[['Region', 'Product_id']].groupby(['Region']).agg('count').sort_values(by='Product_id', ascending=False)

,Product_id
Region,
South,10791885
West,8292913
Midwest,7597325
Northeast,5722736


Again we see that the South has sold the most products and the Northeast has sold the least. 

In [19]:
# checking what region customers spend the most in sales
df2[['prices', 'Region']].groupby('Region').agg('sum').sort_values(by ='prices', ascending=False)

,prices
Region,
South,131613265.4
Midwest,96603658.7
West,93874398.7
Northeast,66126199.8


Southern customers were the top total spenders and Northeastern customers were the least.

In [20]:
# Use crosstab to see the frequency of low and high spending customers in each region 
cross = pd.crosstab(df2['Region'], df2['Spending_flag'], dropna=False).sort_index()
cross

Spending_flag,High spender,Low spender
Region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


The South has the highest frequency of High spending customers. The Northeast has the least. 

In [21]:
# Use crosstab to see the customer frequency per region
cross2 = pd.crosstab(df2['Region'], df2['Frequency_flag'], dropna=False).sort_index()
cross2

Frequency_flag,Frequent customer,Non-frequent customer,Regular customer
Region,,,
Midwest,5017709,863419,1716197
Northeast,3804566,630182,1287988
South,7192372,1215088,2384420
West,5545206,927748,1819959


The South also has the most "frequent customers". The Northeast has the least "frequent customers". 


In [22]:
# Use crosstab to see if there are more loyal customers in different regions
cross3 = pd.crosstab(df2['Region'], df2['Loyalty_flag'], dropna=False).sort_index()
cross3

Loyalty_flag,Loyal customer,New customer,Regular customer
Region,,,
Midwest,2373774,1472573,3750978
Northeast,1841785,1100207,2780744
South,3405844,2074410,5311631
West,2662690,1596800,4033423


The South has the most loyal customers, and the Northeast has the least. 

# Create  dataframe to show aggregate values for regions

In [23]:
# create a new df that summarizes data by region
# rename the columns so they are clear
df_region = df2[['prices', 'Region', 'Customer_id', 'Order_id']].groupby(['Region']).agg(
    {'prices':'sum', 'Customer_id': pd.Series.nunique, 'Order_id': pd.Series.nunique})
df_region = df_region.rename(columns={'prices':'Prices_sum', 'Customer_id': 'User_count', 'Order_id': 'Order_count'})
df_region

,Prices_sum,User_count,Order_count
Region,,,
Midwest,96603658.7,48519,754275
Northeast,66126199.8,36388,567162
South,131613265.4,68737,1071562
West,93874398.7,52565,821686


In [24]:
# create calculated columns based on the aggregated data
df_region['Price_per_order'] = df_region['Prices_sum']/df_region['Order_count']
df_region['Price_per_user'] = df_region['Prices_sum']/df_region['User_count']
df_region['Order_per_user'] = df_region['Order_count']/df_region['User_count']

In [25]:
df_region.sort_values(by='Price_per_user', ascending=False)

,Prices_sum,User_count,Order_count,Price_per_order,Price_per_user,Order_per_user
Region,,,,,,
Midwest,96603658.7,48519,754275,128.074852,1991.048016,15.545972
South,131613265.4,68737,1071562,122.823752,1914.736829,15.589304
Northeast,66126199.8,36388,567162,116.591379,1817.252935,15.586512
West,93874398.7,52565,821686,114.246073,1785.872704,15.631808


# Excluding low frequency customers 

In [26]:
# Create variable for # of orders made by each user.The count of user_id would be the number of orders that user has made 

df2['Total_orders']=df2.groupby(['Customer_id']).Customer_id.transform('count')

In [27]:
# Create exclusion flag by counting the amount of times Customer_id shows up in dataframe, less than 5 is low activity
results_2=[]
for total in df2['Total_orders']:
    if total < 5:
        results_2.append('Low Activity')
    else:
        results_2.append('Regular Activity')

In [28]:
# Include column in dataframe
df2['Activity_flag']=results_2

In [29]:
# Check how many low activity customers there are 
df2['Activity_flag'].value_counts(dropna=False)

Regular Activity    32401975
Low Activity            2884
Name: Activity_flag, dtype: int64

In [30]:
# Create subset of low activity customers
low_activity_customers= df2[df2['Activity_flag']=='Low Activity']

In [31]:
# Check subset to make sure successful 
low_activity_customers.shape

(2884, 31)

In [32]:
# Export low activity subset
low_activity_customers.to_csv(os.path.join(path, 'Data','Prepared Data', 'low_activity_customers.csv'))

In [33]:
# Exclude sample from dataset
df2.drop(df2[df2['Activity_flag']=='Low Activity'].index, inplace=True)

In [34]:
# Check to make sure successful 
df2['Activity_flag'].value_counts()

Regular Activity    32401975
Name: Activity_flag, dtype: int64

In [35]:
# Viewing dataset
pd.set_option('display.max_columns', None)
df2.head()

,Order_id,Customer_id,Order_number,Orders_day_of_week,Order_time_of_day,Days_since_prior_order,Product_id,Add_to_cart_order,Reordered,Product_name,department_id,prices,Busiest_day,Busiest_2_days,Busiest_period_of_day,Max_order,Loyalty_flag,Mean_order_price,Spending_flag,Order_frequency,Frequency_flag,Gender,State,Age,Date_joined,#_of_dependants,Marital_status,Income,Region,Total_orders,Activity_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,7,9.0,Regularly_busy,Regularly_busy,Fewest Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
1,2398795,1,2,3,7,15.0,196,1,1,Soda,7,9.0,Regularly_busy,Least_busy_days,Fewest Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
2,473747,1,3,3,12,21.0,196,1,1,Soda,7,9.0,Regularly_busy,Least_busy_days,Most Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
3,2254736,1,4,4,7,29.0,196,1,1,Soda,7,9.0,Least_busy,Least_busy_days,Fewest Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity
4,431534,1,5,4,15,28.0,196,1,1,Soda,7,9.0,Least_busy,Least_busy_days,Most Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,59,Regular Activity


In [36]:
# Create subset of regular activity customers
df3= df2[df2['Activity_flag']=='Regular Activity']

In [2]:
# Export regular activity subset
df3.to_pickle(os.path.join(path, 'Data','Prepared Data', 'reg_activity_customers.pkl'))